# Retrieval Augmented Generation with Amazon Bedrock - Retrieving Data Automatically from APIs

> *PLEASE NOTE: This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

Another important type of retrieval which customers can take advantage of with LLM is **structured data retrieval** from APIs. Structured data retrieval is extremely useful for augmenting LLM applications with up to date information which can be retrieved in a repeatable manner, but the outputs are always changing. An example of a question you might ask an LLM which uses this type of retrieval might be "How long will it take for my Amazon.com order containing socks to arrive?". In this notebook, we will show how to integrate an LLM with a backend API service which has the ability to answer a user's question through RAG.

Specifically, we will be building a tool which is able to tell you the weather based on natural language. This is a fairly trivial example, but it does a good job of showing how multiple API tools can be used by an LLM to retrieve dynamic data to augment a prompt. Here is a visual of the architecture we will be building today.

![api](./images/api.png)

Let's get started!

---
## Setup `boto3` Connection

In [24]:
!pip install pydantic==1.10.14 --force-reinstall

  Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (150 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.17.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.17.1 requires dash-html-components

In [25]:
import boto3
import os
from IPython.display import Markdown, display, Pretty

region = os.environ.get("AWS_REGION")
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

---
## Defining the API Tools

The first thing we need to do for our LLM is define the tools it has access to. In this case we will be defining local Python functions, but it important to not that these could be any type of application service. Examples of what these tools might be on AWS include...

* An AWS Lambda function
* An Amazon RDS database connection
* An Amazon DynamnoDB table
  
More generic examples include...

* REST APIs
* Data warehouses, data lakes, and databases
* Computation engines

In this case, we define two tools which reach external APIs below with two python functions
1. the ability to retrieve the latitude and longitude of a place given a natural language input
2. the ability to retrieve the weather given an input latitude and longitude

#### ^^^ The API for getting the latitude is an open source and so we might encounter a throtlling by then ^^^

In [26]:
!pip install geopy -q

In [27]:
import requests
from geopy.geocoders import Nominatim

def get_weather(latitude: str, longitude: str) -> dict:
    """
    Checking the weather using the open-meteo API
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

def get_lat_long(place: str) -> dict:
    """
    Geocoding using the Nominatim API
    """
    app = Nominatim(user_agent="tutorial")
    location = app.geocode(place).raw

    return {"latitude": location['lat'], "longitude": location['lon']}

def call_function(tool_name, parameters):
    func = globals()[tool_name]
    output = func(**parameters)
    return output

We also define a function called `call_function` which is used to abstract the tool name. You can see an example of determining the weather in Las Vegas below.

In [28]:
place = 'Las Vegas'
lat_long_response = call_function('get_lat_long', {'place' : place})
weather_response = call_function('get_weather', lat_long_response)
print(f'Weather in {place} is...')
weather_response

Weather in Las Vegas is...


{'latitude': 36.16438,
 'longitude': -115.14392,
 'generationtime_ms': 0.07796287536621094,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 620.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-09-15T16:15',
  'interval': 900,
  'temperature': 30.6,
  'windspeed': 15.9,
  'winddirection': 175,
  'is_day': 1,
  'weathercode': 0}}

As you might expect, we have to describe our tools to our LLM, so it knows how to use them. The strings below describe the python functions for lat/long and weather to Claude in an XML friendly format which we have seen previously in the workshop.

In [29]:
get_weather_description = """\
<tool_description>
<tool_name>get_weather</tool_name>
<parameters>
<name>latitude</name>
<name>longitude</name>
</parameters>
</tool_description>
"""

get_lat_long_description = """
<tool_description>
<tool_name>get_lat_long</tool_name>
<parameters>
<name>place</name>  
</parameters>
</tool_description>"""

list_of_tools_specs = [get_weather_description, get_lat_long_description]
tools_string = ''.join(list_of_tools_specs)
print(tools_string)

<tool_description>
<tool_name>get_weather</tool_name>
<parameters>
<name>latitude</name>
<name>longitude</name>
</parameters>
</tool_description>

<tool_description>
<tool_name>get_lat_long</tool_name>
<parameters>
<name>place</name>  
</parameters>
</tool_description>


---
## Define Prompts to Orchestrate our LLM Using Tools

Now that the tools are defined both programmatically and as a string, we can start orchestrating the flow which will answer user questions. The first step to this is creating a prompt which defines the rules of operation for Claude. In the prompt below, we provide explicit direction on how Claude should use tools to answer these questions.

In [30]:
from langchain import PromptTemplate

TOOL_TEMPLATE = """\
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. Only invoke one function at a time and wait for the results before invoking another function.
4. The Results of the function will be in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem?

Assistant:
"""
TOOL_PROMPT = PromptTemplate.from_template(TOOL_TEMPLATE)

---
## Executing the Workflow

Armed with our prompt and structured tools, we can now write an orchestration function which will iteratively step through the logical tasks to answer a user question. In the cell below we use the `invoke_model` function to generate a response with Claude and the `single_retriever_step` function to iteratively call tools when the LLM tells us we need to. The general flow works like this...

1. The user enters an input to the application
2. The user input is merged with the original prompt and sent to the LLM to determine the next step
3. If the LLM knows the answer, it will answer and we are done. If not, go to next step 4.
4. The LLM will determine which tool to use to answer the question.
5. We will use the tool as directed by the LLM and retrieve the results.
6. We provide the results back into the original prompt as more context.
7. We ask the LLM the next step or if knows the answer.
8. Return to step 3.

If this is a bit confusing do not panic, we will walk through this flow in an example shortly!

In [31]:
!pip install xmltodict

In [32]:
import xmltodict
import json

def invoke_model(prompt):
    client = boto3.client(service_name='bedrock-runtime', region_name=os.environ.get("AWS_REGION"),)
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500, "temperature": 0,})
    modelId = "anthropic.claude-instant-v1"
    response = client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    return json.loads(response.get("body").read()).get("completion")

def single_retriever_step(prompt, output):
    # first check if the model has answered the question
    done = False
    if '<answer>' in output:
        answer = output.split('<answer>')[1]
        answer = answer.split('</answer>')[0]
        done = True
        return done, answer
    
    # if the model has not answered the question, go execute a function
    else:

        # parse the output for any 
        function_xml = output.split('<function_calls>')[1]
        function_xml = function_xml.split('</function_calls>')[0]
        function_dict = xmltodict.parse(function_xml)
        func_name = function_dict['invoke']['tool_name']
        parameters = function_dict['invoke']['parameters']

        # call the function which was parsed
        func_response = call_function(func_name, parameters)

        # create the next human input
        func_response_str = '\n\nHuman: Here is the result from your function call\n\n'
        func_response_str = func_response_str + f'<function_results>\n{func_response}\n</function_results>'
        func_response_str = func_response_str + '\n\nIf you know the answer, say it. If not, what is the next step?\n\nAssistant:'

        # augment the prompt
        prompt = prompt + output + func_response_str
    return done, prompt

Let's start our first example `What is the weather in Las Vegas?`. The code below asks the LLM what the first step is and you will notice that the LLM is able to ascertain it needs to use the `get_lat_long` tool first.

In [33]:
user_input = 'What is the weather in Las Vegas?'

next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

output = invoke_model(next_step).strip()
done, next_step = single_retriever_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

The first step is to invoke the get_lat_long function to get the latitude and longitude of Las Vegas:

<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>  
<parameters>
<place>Las Vegas</place>
</parameters>
</invoke>
</function_calls>

Great, Claude has figured out that we should first call the lat and long tool. The next step is then orchestrated just like the first. This time, Claude uses the lat/long from the first request to now ask for the weather of that specific location.

In [34]:
output = invoke_model(next_step).strip()
done, next_step = single_retriever_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

The next step is to invoke the get_weather function using the latitude and longitude returned from the previous function call to get the weather in Las Vegas:

<function_calls>
<invoke>  
<tool_name>get_weather</tool_name>
<parameters>
<latitude>36.1672559</latitude>
<longitude>-115.148516</longitude>  
</parameters>
</invoke>
</function_calls>

Finally the LLM is able to answer the question based on the input function above. Very cool!

In [35]:
output = invoke_model(next_step).strip()
done, next_step = single_retriever_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

Final answer from LLM:

The weather in Las Vegas is 30.6 degrees Celsius with 15.9 km/h winds from the southeast.


Let's try another example to show how a different place (Singapore) can be used in this example. Notice how we set the for loop to 5 iterations even though the model only uses 3 of these. This iteration capping is common in agent workflows and should be tuned according to your use case. 

In [36]:
user_input = 'What is the weather in Singapore?'
next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

for i in range(5):
    output = invoke_model(next_step).strip()
    done, next_step = single_retriever_step(next_step, output)
    if not done:
        display(Pretty(f'{output}'))
    else:
        display(Pretty('Final answer from LLM:\n'+f'{next_step}'))
        break

The first step is to invoke the get_lat_long function to get the latitude and longitude of Singapore, since that information is needed by the get_weather function to return the weather.

<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>  
<parameters>
<place>Singapore</place>
</parameters>
</invoke>
</function_calls>

The next step is to invoke the get_weather function, passing in the latitude and longitude retrieved from the previous get_lat_long call, to get the weather in Singapore.

<function_calls>
<invoke>  
<tool_name>get_weather</tool_name>
<parameters>
<latitude>1.357107</latitude>
<longitude>103.8194992</longitude>  
</parameters>
</invoke>
</function_calls>

Final answer from LLM:

The weather in Singapore is 27.0 degrees Celsius, with a wind speed of 5.1 km/h from the southeast (184 degrees) and overcast skies (weather code 3).


In [37]:
user_input = 'What is the weather in Chandigarh, India?'
next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

for i in range(5):
    output = invoke_model(next_step).strip()
    done, next_step = single_retriever_step(next_step, output)
    if not done:
        display(Pretty(f'{output}'))
    else:
        display(Pretty('Final answer from LLM:\n'+f'{next_step}'))
        break

The first step is to invoke the get_lat_long function to get the latitude and longitude of Chandigarh, India since that information is needed by the get_weather function.

<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>  
<parameters>
<place>Chandigarh, India</place>
</parameters>
</invoke>
</function_calls>

The next step is to invoke the get_weather function using the latitude and longitude retrieved from the previous function call to get the weather in Chandigarh, India.

<function_calls>
<invoke>  
<tool_name>get_weather</tool_name>
<parameters>
<latitude>30.7334421</latitude> 
<longitude>76.7797143</longitude>
</parameters>
</invoke>
</function_calls>

Final answer from LLM:

The weather in Chandigarh, India is 26.5 degrees Celsius, with a wind speed of 1.8 km/h blowing from the east.


---
## Leveraging Open Source Agents with Bedrock

As with the previous notebook, there are many ways to orchestrate agent workflows with LLMs. One of the popular open source frameworks for this is [LangChain's agent module](https://python.langchain.com/docs/modules/agents/). LangChain provides a variety of agent types as well as the same [integration to Amazon Bedrock](https://python.langchain.com/docs/integrations/llms/bedrock) being available for these agent based workflows. Let's work on implementing the same workflow as above into the LangChain ecosystem.

First, we can define our LLM with the `Bedrock` class.

In [38]:
from langchain.llms.bedrock import Bedrock

agent_llm = Bedrock(
    client=boto3.client(service_name='bedrock-runtime', region_name=os.environ.get("AWS_REGION"),),
    model_id="anthropic.claude-instant-v1",
    model_kwargs={
        "max_tokens_to_sample": 500,
        "temperature": 0.0,
    },
)

In [39]:
from langchain.tools import tool
import ast

@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """
    Geocoding using the Nominatim API
    """
    app = Nominatim(user_agent="tutorial")
    location = app.geocode(place).raw

    return {"latitude": location['lat'], "longitude": location['lon']}

@tool ("get_weather_dict")
def get_weather_dict(co_ord_str: str) -> dict:
    """
    Returns weather data for a given a dictionary having latitude and longitude.
    """
    if '{' in co_ord_str:
        co_ord_str = ast.literal_eval(co_ord_str)
        latitude = co_ord_str['latitude']
        longitude = co_ord_str['longitude']
    else:
        latitude = float(co_ord_str.split(",")[0].strip())
        longitude = float(co_ord_str.split(",")[1].strip())
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

As well as being able to define our own tools, LangChain has some pre-built tools which are general purpose. One well documented issue with LLMs is that they are not good at arithmetic only from natural language. We can use the `LLMMathChain` from LangChain to help solve this problem. In this chain, we are able to ask an LLM simple math questions and the question will be sent to a calculator in order to return a perfect mathematical result. An example is included below.

In [40]:
!pip install numexpr

In [41]:
from langchain import LLMMathChain

math_tool_template = """\
Given a question from the Human with a math problem, provide only a single line mathematical expression that solves the problem in the following format.
Never solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

llm_math_chain = LLMMathChain.from_llm(llm=agent_llm, verbose=True)
llm_math_chain.llm_chain.prompt.template = math_tool_template
result = llm_math_chain('What is nine times 465?')



> Entering new LLMMathChain chain...
What is nine times 465? ```text
9 * 465
```
Answer: 4185
> Finished chain.


Now that all the tools are defined, lets create a [ReACT agent](https://python.langchain.com/docs/modules/agents/agent_types/react) (Reason + Act) which will leverage the LLM to generate the next set of actions to solve a given question. This next cell creates the agent from a list of tools we provide.

In [42]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools_list = [get_lat_long, get_weather_dict]
tools_list.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)
react_agent = initialize_agent(
    tools_list, 
    agent_llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

The ReAct framework also expects a specific format to be followed by our LLM. This custom prompt adds place holders for the 3 components which need to be followed.

1. **Action** which is the tool to use
2. **Action Input** is the parameters coming from the LLM which will go into the tool or function to be called
3. **Observation** is the final result of the call from the tool

These three components to ReAct are recursively executed until there is an answer to the question or we reach the limit of execution steps.

In [43]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather_dict", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

Now we can send in the first question just like the one we used in the DIY system. Notice how the logging allows you to see how the LLM is reasoning through the task within the ReAct framework.

In [44]:
result = react_agent("Can you check the weather in Las Vegas for me?")



> Entering new AgentExecutor chain...
 Here is the response in the specified format:

Question: Can you check the weather in Las Vegas for me?
Thought: I need to get the latitude and longitude of Las Vegas first
Action: get_lat_long 
Action Input: Las Vegas
Observation: {'latitude': '36.1672559', 'longitude': '-115.148516'}
Thought: Now I can use the latitude and longitude to get the weather details
Action: get_weather_dict
Action Input: 36.1672559, -115.148516
Observation: {'latitude': 36.16438, 'longitude': -115.14392, 'generationtime_ms': 0.07402896881103516, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 620.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-09-15T16:15', 'interval': 900, 'temperature': 30.6, 'windspeed': 15.9, 'winddirection': 175, 'is_day': 1, 'weathercode': 0}}


Another example is shown below.

In [45]:
result = react_agent("can you check the weather in Seattle WA for me?")



> Entering new AgentExecutor chain...
 Here is the response in the specified format:

Question: can you check the weather in Seattle WA for me?
Thought: I need to get the latitude and longitude of Seattle first
Action: get_lat_long 
Action Input: Seattle WA
Observation: {'latitude': '47.6038321', 'longitude': '-122.330062'}
Thought: Now I can use the latitude and longitude to get the weather details
Action: get_weather_dict
Action Input: 47.6038321, -122.330062
Observation: {'latitude': 47.595562, 'longitude': -122.32443, 'generationtime_ms': 0.051975250244140625, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 40.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-09-15T16:15', 'interval': 900, 'temperature': 13.4, 'windspeed': 11.7, 'winddirection': 11, 'is_day': 1, 'weathercode': 1}}


In this last example, we will give a different type of problem to the LLM where it will be able to take advantage of the math tool as well as the lat/long tool instead of only talking about weather.

In [46]:
result = react_agent("what is the latitude of Washington DC multiplied by 2")



> Entering new AgentExecutor chain...
 Here is the response in the specified format:

Question: what is the latitude of Washington DC multiplied by 2

Thought: I need to first get the latitude of Washington DC
Action: get_lat_long 
Action Input: Washington DC
Observation: {'latitude': '38.8950368', 'longitude': '-77.0365427'}
Thought: Now I need to multiply the latitude by 2
Action: Calculator
Action Input: 38.8950368 * 2

> Entering new LLMMathChain chain...
38.8950368 * 2 ```text
38.8950368 * 2
```
Answer: 77.7900736
> Finished chain.

Observation: Answer: 77.7900736
Thought: I now know the final answer
Final Answer: 77.7900736

> Finished chain.


---
## Next steps

Now that you have used a few different retrieval systems, lets move on to the next notebook where you can apply the skills you've learned so far!